In [61]:
import pandas
from arcgis import GeoAccessor

In [62]:
#File paths
stem_path = r'C:/Users/tjjoh/Documents/GIS5571'
gdb_path = stem_path + r'/Lab3_GDB.gdb'

fc_path = gdb_path + r'/ThirtyDayTemperature'

idw_max_path = gdb_path + r'/IDW_max'
idw_min_path = gdb_path + r'/IDW_min'
idw_avg_path = gdb_path + r'/IDW_avg'

natn_max_path = gdb_path + r'/NatN_max'
natn_min_path = gdb_path + r'/NatN_min'
natn_avg_path = gdb_path + r'/NatN_avg'

krig_max_path = gdb_path + r'/Krig_max'
krig_min_path = gdb_path + r'/Krig_min'
krig_avg_path = gdb_path + r'/Krig_avg'

max_layer = stem_path + r'/GIS5571/Lab3/max_style.lyrx'
min_layer = stem_path + r'/GIS5571/Lab3/min_style.lyrx'
avg_layer = stem_path + r'/GIS5571/Lab3/avg_style.lyrx'

In [63]:
#Create file geodatabase
arcpy.management.CreateFileGDB(
    out_folder_path = stem_path,
    out_name = 'Lab3_GDB',
    out_version = 'CURRENT'
)

<Result 'C:/Users/tjjoh/Documents/GIS5571/Lab3_GDB.gdb'>

In [64]:
#url to retrieve last 30 days of weather data from NDAWN
url = r'https://ndawn.ndsu.nodak.edu/table.csv?station>0&variable=ddavt&variable=ddmxt&variable=ddmnt&variableyear=2024&ttype=daily&quick_pick=30_d&begin_date=2024-11-11&end_date=2024-11-11'

In [65]:
#Read data with pandas
data = pandas.read_csv(url, skiprows = [0,1,2,4])

In [66]:
#Aggregate the data to find the highest high, lowest low, and overall average for each station
summarized_data = data.groupby('Station Name').agg({'Latitude': 'mean', 'Longitude': 'mean', 'Day': 'count', 'Max Temp': 'max', 'Min Temp': 'min', 'Avg Temp': 'mean'}).reset_index()

In [67]:
#Convert the aggregated data to a spatially enabled dataframe
data_sedf = GeoAccessor.from_xy(summarized_data, x_column = 'Longitude', y_column = 'Latitude', sr = 4326)

In [68]:
#Remove weather stations that are missing data
data_sedf = data_sedf[data_sedf['Day'] == 30]

In [69]:
#Convert data to a feature class
data_sedf.spatial.to_featureclass(location = fc_path)

'C:\\Users\\tjjoh\\Documents\\GIS5571\\Lab3_GDB.gdb\\ThirtyDayTemperature'

**IDW**

In [70]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW_max = arcpy.sa.Idw(
        in_point_features = fc_path,
        z_field = 'max_temp',
        cell_size = 0.021010908,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features=None
    )
    IDW_max.save(idw_max_path)

In [71]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW_min = arcpy.sa.Idw(
        in_point_features = fc_path,
        z_field = 'min_temp',
        cell_size = 0.021010908,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features=None
    )
    IDW_min.save(idw_min_path)

In [72]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    IDW_avg = arcpy.sa.Idw(
        in_point_features = fc_path,
        z_field = 'avg_temp',
        cell_size = 0.021010908,
        power = 2,
        search_radius = 'VARIABLE 12',
        in_barrier_polyline_features=None
    )
    IDW_avg.save(idw_avg_path)

**Natural Neighbor**

In [73]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    NatN_max = arcpy.sa.NaturalNeighbor(
        in_point_features = 'ThirtyDayTemperature',
        z_field = 'max_temp',
        cell_size = 0.021010908
    )
    NatN_max.save(natn_max_path)

In [74]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    NatN_min = arcpy.sa.NaturalNeighbor(
        in_point_features = 'ThirtyDayTemperature',
        z_field = 'min_temp',
        cell_size = 0.021010908
    )
    NatN_min.save(natn_min_path)

In [75]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    NatN_avg = arcpy.sa.NaturalNeighbor(
        in_point_features = 'ThirtyDayTemperature',
        z_field = 'avg_temp',
        cell_size = 0.021010908
    )
    NatN_avg.save(natn_avg_path)

**Kriging**

In [76]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Krig_max = arcpy.sa.Kriging(
        in_point_features = fc_path,
        z_field = 'max_temp',
        kriging_model = 'Spherical # # # #',
        cell_size = 0.021010908,
        search_radius = 'VARIABLE 12',
        out_variance_prediction_raster = None
    )
    Krig_max.save(krig_max_path)

In [77]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Krig_min = arcpy.sa.Kriging(
        in_point_features = fc_path,
        z_field = 'min_temp',
        kriging_model = 'Spherical # # # #',
        cell_size = 0.021010908,
        search_radius = 'VARIABLE 12',
        out_variance_prediction_raster = None
    )
    Krig_min.save(krig_min_path)

In [78]:
with arcpy.EnvManager(scratchWorkspace = gdb_path):
    Krig_avg = arcpy.sa.Kriging(
        in_point_features = fc_path,
        z_field = 'avg_temp',
        kriging_model = 'Spherical # # # #',
        cell_size = 0.021010908,
        search_radius = 'VARIABLE 12',
        out_variance_prediction_raster = None
    )
    Krig_avg.save(krig_avg_path)

**Standardize Symbology**

In [79]:
project = arcpy.mp.ArcGISProject('CURRENT')

In [80]:
map = project.listMaps("Map")[0]
max_layers = ['IDW_max', 'NatN_max', 'Krig_max']
min_layers = ['IDW_min', 'NatN_min', 'Krig_min']
avg_layers = ['IDW_avg', 'NatN_avg', 'Krig_avg']

In [81]:
for layer in map.listLayers():
    if layer.name in max_layers:
        symbology_layer = arcpy.mp.LayerFile(max_layer).listLayers()[0]
        layer.symbology = symbology_layer.symbology
    if layer.name in min_layers:
        symbology_layer = arcpy.mp.LayerFile(min_layer).listLayers()[0]
        layer.symbology = symbology_layer.symbology
    if layer.name in avg_layers:
        symbology_layer = arcpy.mp.LayerFile(avg_layer).listLayers()[0]
        layer.symbology = symbology_layer.symbology